In [21]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Read data into a dataframe and 
# df = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = df.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]
# densityPlot = sns.kdeplot(counts, bw_adjust=3)
# yLabels = densityPlot.get_yticks()
# densityPlot.set_yticklabels('{:,.0%}'.format(y) for y in yLabels)
# plt.xlabel("Number of reviews")
# plt.ylabel("Percentage of users")
# plt.savefig(fname="densityPlot")
# plt.show()

In [22]:
# import pandas as pd

# # Read ratings into a dataframe and narrow down reviewers to desired review count
# ratings = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = ratings.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]

# # Create and apply mask that removes all other reviewers.
# mask = ratings['reviewerID'].isin(counts.index)
# ratings = ratings[mask]

# # Read metadata into dataframe and define asins that are used by making a set
# metadata = pd.read_csv("metadata.csv")
# used_asins = set(ratings["asin"])

# # Create and apply mask that removes unreferenced items
# mask = metadata["asin"].isin(used_asins)
# metadata = metadata[mask]
# metadata.to_pickle("metadata.pickle")
# ratings.to_pickle("ratings.pickle")


In [23]:
# import pandas as pd
# import numpy as np
# from scipy.sparse import csr_matrix, save_npz
# ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
# metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
# ratings = ratings[["reviewerID", "asin", "overall"]]

# # Series of each unique value of reviewerID and asin 
# reviewerIDs = ratings['reviewerID'].unique()
# asins = ratings["asin"].unique()
# # Dictionary that maps reviewers to an index
# reviewerMap = {reviewerID: i for i, reviewerID in enumerate(reviewerIDs)}

# # Dictionary that maps asins to an index
# asinMap = {asin: i for i, asin in enumerate(asins)}

# # Initialize the utility matrix with zeros
# utilityMatrixArray = np.zeros((len(reviewerIDs), len(asins)), "int8")

# # Fill in the utility matrix with data from the records array
# for row in ratings.itertuples():
#     reviewerIndex = reviewerMap[row.reviewerID]
#     asinIndex = asinMap[row.asin]
#     utilityMatrixArray[reviewerIndex, asinIndex] = row.overall
# utilityMatrix = csr_matrix(utilityMatrixArray)
# save_npz("utilityMatrix.npz", utilityMatrix)

In [24]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
utilityMatrix = load_npz("utilityMatrix.npz")

# Compute the pairwise cosine similarity between users
user_similarity = cosine_similarity(utilityMatrix, dense_output=False)

# Create an anonymous user with four reviews
anonymous_user = pd.DataFrame({'item_id': [1, 2, 3, 4], 'rating': [4, 3, 5, 2]})

# Compute the similarity between the anonymous user and every other user
user_similarity_scores = cosine_similarity(utilityMatrix, anonymous_user.set_index('item_id').T.fillna(0))

# Get the k most similar users to the anonymous user
k = 10
similar_user_indices = np.argsort(user_similarity_scores)[0][-k:]

# Compute the predicted ratings for the anonymous user
predicted_ratings = utilityMatrix.iloc[similar_user_indices].mean().sort_values(ascending=False)

# Recommend the top item
top_item = predicted_ratings.index[0]

# ratings = ratings.rename(columns={"reviewerID": "user_id", "asin" : "item_id", "overall": "rating"})
# # Define a Reader object to parse the dataframe into the Surprise dataset format
# reader = Reader(rating_scale=(1, 5))
# data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

# # Split the data into training and testing datasets
# train_data, test_data = train_test_split(data, test_size=0.2)

# # Use KNNWithMeans algorithm to fit the training dataset
# sim_options = {'name': 'cosine', 'user_based': True} # similarity metric and user-based filtering
# algo = KNNWithMeans(k=50, sim_options=sim_options)
# algo.fit(train_data)

# # Evaluate the model's accuracy on the testing data
# predictions = algo.test(test_data)
# accuracy.rmse(predictions)

# # Predict ratings for the anonymous user
# user_ratings = pd.DataFrame({'user_id': [0, 0, 0, 0], 'item_id': [1, 2, 3, 4], 'rating': [4, 3, 5, 2]})
# user_data = Dataset.load_from_df(user_ratings, reader)
# user_predictions = algo.test(user_data.build_full_trainset().build_anti_testset())

# # Recommend the top item
# top_item = max(user_predictions, key=lambda x: x.est).iid



# # Create training and testing datasets
# trainingData = utilityMatrix[:113404]
# testingData = utilityMatrix[113404:]

# # X_train, X_test= train_test_split(utilityMatrix, test_size=0.2, random_state=42)
# model = TruncatedSVD(n_components=50, random_state=42).fit(trainingData)
# resultant_matrix = model.transform(trainingData)
# var_explained = model.explained_variance_ratio_.sum()
# var_explained

AttributeError: fillna not found